# **Capstone Project: Starbucks Regional Review Intelligence using IBM Granite AI**


## **Background**
Customer reviews are one of the main sources of organic feedback that can reflect users' real experiences with Starbucks products and services. With reviews coming from various locations and regions, as a data analyst, you must be able to help managers understand patterns of customer satisfaction and complaints based on review data from various regions, in order to optimize service and product quality and increase Starbucks customer retention.

## **Project Objectives**
1. Identify customer sentiment patterns based on regional & cultural context
2. Extract complaints and highlight dominant issue patterns for strategic decision-making
3. Generate an actionable, impactful regional improvement roadmap that maximizes ROI (Return on Investment).

## **Raw Dataset**
- https://www.kaggle.com/datasets/harshalhonde/starbucks-reviews-dataset

## **Dataset Overview**
Dataset was cut down to only 3 state with each of maximum 100 reviews to handle the limited resource of the credits given by hactiv8 team, but this project serve on how LLM can handle in data analysis from reviews

## **Data Preparation**

- Load and clean review dataset.
- Extract state from location.
- Clean review text.
- Sample balanced data for 3 target states.

This ensure the data is ready for further use

In [48]:
import pandas as pd
import numpy as np
import re
import nltk
import unicodedata

nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
# Load dataset
df = pd.read_csv('reviews_data.csv')

# Check few first rows
df.head()

# Drop rows with missing info
initial_shape = df.shape
df.dropna(subset=['Review', 'location'], inplace=True)

In [50]:
#
def clean_review_text(text):
    """Clean and normalize review text"""
    if not isinstance(text, str):
        return ""

    text = re.sub(r'\b(not|no)\s+(\w+)', r'\1_\2', text, flags=re.IGNORECASE)

    text = re.sub(r'starbucks|sbux|http\S+', '', text, flags=re.IGNORECASE)

    text = re.sub(r'[^\w\s]', '', text)

    return text.strip().lower()

In [51]:
def extract_state(location_string):
    if isinstance(location_string, str):
        match = re.search(r',\s*([A-Z]{2})$', location_string)
        if match:
            return match.group(1)
    return None

df['state'] = df['location'].apply(extract_state)

In [52]:
target_states = ['CA', 'FL', 'TX']

df_filtered = df[df['state'].isin(target_states)].copy()

In [53]:
sampled_df = (
    df_filtered
    .groupby('state')
    .apply(lambda x: x.sample(n=min(len(x), 100), random_state=42))
    .reset_index(drop=True)
)

print("Sampled data shape:", sampled_df.shape)
print(sampled_df['state'].value_counts())


Sampled data shape: (186, 7)
state
CA    100
FL     46
TX     40
Name: count, dtype: int64


/tmp/ipython-input-2604270383.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 100), random_state=42))


In [54]:
# Clean review
sampled_df['clean_review'] = sampled_df['Review'].apply(clean_review_text)
sampled_df['review_length'] = sampled_df['clean_review'].str.len()
sampled_df['word_count'] = sampled_df['clean_review'].str.split().str.len()

sampled_df.to_csv("cleaned_sampled_reviews.csv", index=False)

In [55]:
# Dataset shape
print("Dataset shape (rows, columns):", sampled_df.shape)

# Count per state
print("\nReview counts per state:")
print(sampled_df['state'].value_counts())

# Descriptive stats
print("\nReview length & word count stats:")
print(sampled_df[['review_length', 'word_count']].describe())

# sample cleaned reviews
print("\nSample cleaned reviews:")
for i, row in sampled_df[['state', 'clean_review']].sample(5, random_state=1).iterrows():
    print(f"[{row['state']}] {row['clean_review'][:100]}...")


Dataset shape (rows, columns): (186, 10)

Review counts per state:
state
CA    100
FL     46
TX     40
Name: count, dtype: int64

Review length & word count stats:
       review_length  word_count
count     186.000000  186.000000
mean      452.698925   86.827957
std       229.797457   44.479482
min        14.000000    2.000000
25%       271.250000   50.250000
50%       450.500000   84.500000
75%       636.750000  120.000000
max       961.000000  189.000000

Sample cleaned reviews:
[CA] i was poisoned by my smoothie after a few hours my stomachache got worse and worse then came 24 hour...
[TX] when you get a good barista you get a good drink but those are hard to come by at  now if you ask th...
[CA] all the money  makes not_only are they ungrateful to their employees but whats the point of being a ...
[CA] retail locations in hollywood california regularly violate customer rights illegally unethically and...
[TX] as soon as he sees me walk in the door he has my croissant warming up thi

## **Regional Behavior Enrichment using IBM Granite**

Each state’s customer behavior is extracted using IBM Granite, the variable are:
- Priority
- Tolerance
- Cultural Norm of Feedback

These will add depth to LLM deciding factor of the reviews

In [56]:
!pip install langchain_community
!pip install replicate

In [57]:
import os
import json
import pandas as pd
from langchain_community.llms import Replicate
from google.colab import userdata

In [58]:
api_token = userdata.get('api_token')

os.environ["REPLICATE_API_TOKEN"] = api_token

model_id = "ibm-granite/granite-3.3-8b-instruct"
llm = Replicate(
    model=model_id,
    replicate_api_token=api_token
)

In [59]:
def generate_regional_context_with_ai(state_code):
    prompt = f"""
As a cultural analyst specializing in consumer behavior in the United States, describe the general characteristics of Starbucks customers in the state of {state_code}. Focus on:

- *Primary Priority:* 1–2 short keywords.
- *Tolerance Level:* 1–2 short keywords.
- *Cultural Norm of Feedback:* 1–2 short keywords.

Respond in JSON like:
{{
  "state": "{state_code}",
  "priority": "short_keyword",
  "tolerance": "short_keyword",
  "cultural_norm": "short_keyword"
}}
""".strip()

    try:
        response = llm.invoke(prompt).strip()
        return json.loads(response)
    except Exception as e:
        print(f"Error generating context for {state_code}: {e}")
        return {
            "state": state_code,
            "priority": "general",
            "tolerance": "medium",
            "cultural_norm": "standard"
        }

In [60]:
unique_states = sampled_df['state'].dropna().unique()

context_list = []

for state in unique_states:
    print(f"Generating context for {state}...")
    context = generate_regional_context_with_ai(state)
    context_list.append(context)
    print(f"{state}: {context}")

# Merge results
regional_context_df = pd.DataFrame(context_list)
sampled_df_with_context = sampled_df.merge(regional_context_df, on='state', how='left')
sampled_df_with_context.to_csv("sampled_df_with_regional_context.csv", index=False)

Generating context for CA...
CA: {'state': 'CA', 'priority': 'Convenience, Quality', 'tolerance': 'Health-conscious, Eco-friendly', 'cultural_norm': 'Digital-feedback, Social-media-influenced'}
Generating context for FL...
FL: {'state': 'FL', 'priority': 'convenience, quality', 'tolerance': 'moderate, adaptable', 'cultural_norm': 'direct, positive'}
Generating context for TX...
TX: {'state': 'TX', 'priority': 'convenience, quality', 'tolerance': 'moderate, adaptable', 'cultural_norm': 'direct, feedback-oriented'}


In [61]:
# Dataset shape
print("Dataset shape (rows, columns):", sampled_df_with_context.shape)

# Count per state
print("\nReview counts per state:")
print(sampled_df_with_context['state'].value_counts())

# Descriptive stats
print("\nReview length & word count stats:")
print(sampled_df_with_context[['review_length', 'word_count']].describe())

# Regional context summary
print("\nRegional context per state:")
print(sampled_df_with_context[['state', 'priority', 'tolerance', 'cultural_norm']].drop_duplicates().to_string(index=False))

# sample cleaned reviews
print("\nSample cleaned reviews:")
for i, row in sampled_df_with_context[['state', 'clean_review']].sample(5, random_state=1).iterrows():
    print(f"[{row['state']}] {row['clean_review'][:100]}...")

Dataset shape (rows, columns): (186, 13)

Review counts per state:
state
CA    100
FL     46
TX     40
Name: count, dtype: int64

Review length & word count stats:
       review_length  word_count
count     186.000000  186.000000
mean      452.698925   86.827957
std       229.797457   44.479482
min        14.000000    2.000000
25%       271.250000   50.250000
50%       450.500000   84.500000
75%       636.750000  120.000000
max       961.000000  189.000000

Regional context per state:
state             priority                      tolerance                             cultural_norm
   CA Convenience, Quality Health-conscious, Eco-friendly Digital-feedback, Social-media-influenced
   FL convenience, quality            moderate, adaptable                          direct, positive
   TX convenience, quality            moderate, adaptable                 direct, feedback-oriented

Sample cleaned reviews:
[CA] i was poisoned by my smoothie after a few hours my stomachache got worse and wor

## **LLM-Based Review Classification**

Using IBM Granite to classify:
- Sentiment (Positive/Negative/Mixed)
- Aspect (Service, Product, etc.)
- Urgency (High/Medium/Low)

with each reviews already given their "summary" by the IBM granite, it's ready for analysis and insight

In [62]:
import pandas as pd
import time
import re
import textwrap
import replicate


df = sampled_df_with_context

In [63]:
def analyze_review_batch_lite(reviews, ratings, states, priorities, tolerances, norms, max_reviews=3):
    short_reviews = [textwrap.shorten(str(r), width=100, placeholder="...") for r in reviews[:max_reviews]]
    ratings = ratings[:max_reviews]
    states = states[:max_reviews]
    priorities = priorities[:max_reviews]
    tolerances = tolerances[:max_reviews]
    norms = norms[:max_reviews]

    review_batch = "\n".join([
        f"[{i}] \"{r}\" | Rating: {rat} | State: {st} | Priority: {prio} | Tolerance: {tol} | Norm: {norm}"
        for i, (r, rat, st, prio, tol, norm) in enumerate(zip(short_reviews, ratings, states, priorities, tolerances, norms))
    ])

    prompt = f"""
You are an AI model that classifies customer reviews based on sentiment, aspect, and urgency.

Respond in this exact format (strictly use pick one!):
[<index>] | Sentiment: <Positive/Negative/Mixed> | Aspect: <Service/Product/Ambiance/Wait Time/Price/Cleanliness/Availability/Unknown> | Urgency: <Low/Medium/High>

Here are the reviews:
{review_batch}
"""

    return safe_replicate_run(prompt)

In [64]:
VALID_ASPECTS = {
    'service': 'Service',
    'product': 'Product',
    'ambiance': 'Ambiance',
    'wait time': 'Wait Time',
    'price': 'Price',
    'cleanliness': 'Cleanliness',
    'availability': 'Availability',
    'unknown': 'Unknown'
}

def normalize_aspect(raw_aspect):
    aspect_cleaned = raw_aspect.strip().lower()

    # Direct match first
    if aspect_cleaned in VALID_ASPECTS:
        return VALID_ASPECTS[aspect_cleaned]

    # Fuzzy/partial matching
    for key in VALID_ASPECTS:
        if key in aspect_cleaned:
            return VALID_ASPECTS[key]

    return 'Unknown'  # Fallback

def parse_batch_output(output_text):
    results = []
    lines = output_text.strip().split("\n")

    for line in lines:
        print("Parsing line:", line)

        match_colon = re.match(
    r"\[<?(\d+)>?\]\s*\|\s*Sentiment:?\s*(\w+)\s*\|\s*Aspect:?\s*([A-Za-z\s\/\-]+)\s*\|\s*Urgency:?\s*(\w+)",
    line, re.IGNORECASE
)
        if match_colon:
            try:
                idx = int(match_colon.group(1))
                sentiment = match_colon.group(2).capitalize()
                raw_aspect = match_colon.group(3)
                aspect = normalize_aspect(raw_aspect)
                urgency = match_colon.group(4).capitalize()

                results.append({
                    'batch_index': idx,
                    'sentiment': sentiment,
                    'aspect': aspect,
                    'urgency': urgency
                })
                continue
            except Exception as e:
                print(f"Error parsing colon format: {e}")

        match_simple = re.match(
    r"\[<?(\d+)>?\]\s*\|\s*(\w+)\s*\|\s*([A-Za-z\s\/\-]+)\s*\|\s*(\w+)",
    line, re.IGNORECASE
)
        if match_simple:
            try:
                idx = int(match_simple.group(1))
                sentiment = match_simple.group(2).capitalize()
                raw_aspect = match_simple.group(3)
                aspect = normalize_aspect(raw_aspect)
                urgency = match_simple.group(4).capitalize()

                results.append({
                    'batch_index': idx,
                    'sentiment': sentiment,
                    'aspect': aspect,
                    'urgency': urgency
                })
            except Exception as e:
                print(f"Error parsing fallback format: {e}")
        else:
            print("Unrecognized format:", line)

    return results

In [65]:
def parse_batch_output_with_retry(output_text, expected_count, max_retries=3):
    for attempt in range(1, max_retries + 1):
        parsed_outputs = parse_batch_output(output_text)

        if len(parsed_outputs) == expected_count:
            print(f"Parsed {len(parsed_outputs)}/{expected_count} successfully")
            return parsed_outputs

        print(f"Retry {attempt}: Parsed {len(parsed_outputs)}/{expected_count}")
        time.sleep(1)

    print(f"Failed to parse after {max_retries} attempts")
    return []


In [66]:
def safe_replicate_run(prompt, retries=2):
    for attempt in range(retries):
        try:
            output = replicate.run(
                "ibm-granite/granite-3.3-8b-instruct",
                input={
                    "prompt": prompt,
                    "temperature": 0.1,
                    "max_new_tokens": 120,
                    "top_p": 0.95
                }
            )
            return "".join(output).strip()
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            time.sleep(3)
    return ""


In [67]:
batch_size = 3
results = []

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]

    print(f"\nProcessing batch {i}-{i + len(batch)}")

    output = analyze_review_batch_lite(
        reviews=batch['clean_review'].tolist(),
        ratings=batch['Rating'].tolist(),
        states=batch['state'].tolist(),
        priorities=batch['priority'].tolist(),
        tolerances=batch['tolerance'].tolist(),
        norms=batch['cultural_norm'].tolist(),
        max_reviews=batch_size
    )

    parsed_outputs = parse_batch_output_with_retry(output, expected_count=len(batch), max_retries=3)

    if parsed_outputs:
        for parsed in parsed_outputs:
            original_idx = batch.iloc[parsed['batch_index']].name
            results.append({
                'original_index': original_idx,
                **parsed
            })
    else:
        print(f"Skipping batch {i}-{i + len(batch)} due to repeated parsing errors.")

    print(f"Processed {min(i + batch_size, len(df))}/{len(df)}")
    time.sleep(2)


Processing batch 0-3
Parsing line: [0] | Sentiment: Mixed | Aspect: Price | Urgency: Low
Parsing line: [1] | Sentiment: Negative | Aspect: Price | Urgency: Low
Parsing line: [2] | Sentiment: Negative | Aspect: Service | Urgency: Medium
Parsed 3/3 successfully
Processed 3/186

Processing batch 3-6
Parsing line: [0] | Sentiment: Negative | Aspect: Service | Urgency: Medium
Parsing line: [1] | Sentiment: Negative | Aspect: Price | Urgency: Low
Parsing line: [2] | Sentiment: Negative | Aspect: Availability | Urgency: Low
Parsed 3/3 successfully
Processed 6/186

Processing batch 6-9
Parsing line: [0] | Sentiment: Negative | Aspect: Ambiance | Urgency: Medium
Parsing line: [1] | Sentiment: Negative | Aspect: Service | Urgency: Medium
Parsing line: [2] | Sentiment: Negative | Aspect: Convenience | Urgency: Low
Parsed 3/3 successfully
Processed 9/186

Processing batch 9-12
Parsing line: [0] | Sentiment: Negative | Aspect: Service | Urgency: Medium
Parsing line: [1] | Sentiment: Negative | Asp

In [68]:
# Merge parsed results into DataFrame
df_results = pd.DataFrame(results)
df_results.set_index('original_index', inplace=True)

df_final = df.copy()
df_final = df_final.join(df_results, how='left')

# Show sample rows
print("Sample of Final Classified Reviews:")
display(df_final[['clean_review', 'sentiment', 'aspect', 'urgency']].head())

# Summary Statistics
print("\nClassification Distribution:")
print("-" * 40)
print("Sentiment counts:\n", df_final['sentiment'].value_counts(dropna=False))
print("\nAspect counts:\n", df_final['aspect'].value_counts(dropna=False))
print("\nUrgency counts:\n", df_final['urgency'].value_counts(dropna=False))

# Missing classification check
missing = df_final[['sentiment', 'aspect', 'urgency']].isna().sum()
if missing.any():
    print("\nMissing values detected:")
    print(missing)
    incomplete = df_final[df_final[['sentiment', 'aspect', 'urgency']].isna().any(axis=1)]
else:
    print("\nAll rows fully classified — no NaNs")

# Save full output
df_final.to_csv("review_analysis_result.csv", index=False)
print("Final result saved to 'review_analysis_result.csv'")


Sample of Final Classified Reviews:


,clean_review,sentiment,aspect,urgency
0,no_review text,Mixed,Price,Low
1,i have only purchased their coffee their price...,Negative,Price,Low
2,i was visiting the at 2183 vista way in ocean...,Negative,Service,Medium
3,in cathedral city ca needs more baristas servi...,Negative,Service,Medium
4,ordered my usual tall iced doppio espresso wit...,Negative,Price,Low



Classification Distribution:
----------------------------------------
Sentiment counts:
 sentiment
Negative    90
NaN         60
Positive    20
Mixed       13
Unknown      2
Neutral      1
Name: count, dtype: int64

Aspect counts:
 aspect
Unknown         63
NaN             60
Service         36
Price           13
Product          9
Availability     2
Ambiance         1
Wait Time        1
Cleanliness      1
Name: count, dtype: int64

Urgency counts:
 urgency
NaN        60
Medium     50
Low        39
High       36
Unknown     1
Name: count, dtype: int64

Missing values detected:
sentiment    60
aspect       60
urgency      60
dtype: int64
Final result saved to 'review_analysis_result.csv'


## **Insights from LLM Analysis**

- What aspects trigger most complaints per state?
- Which aspects are most urgent?
- Common keywords in negative/mixed reviews
- and Actionable Recommendations

Each summary is processed and with the end goal of clearly tell us as the "manager" of what to do for each state the best course of action


In [69]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def extract_top_keywords(texts, top_n=5):
    all_words = ' '.join(texts).lower().split()
    filtered = [word for word in all_words if word not in stop_words and len(word) > 2]
    common = Counter(filtered).most_common(top_n)
    return [word for word, _ in common]



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
def generate_insights(df_combined):
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.switch_backend('Agg')
    print("\n--- Generating Insights and Visualizations ---")

    # Prepare complaint data
    complaints = df_combined[
        df_combined['sentiment'].isin(['Negative', 'Mixed'])
    ].copy()

    complaints['aspect'] = complaints['aspect'].fillna('Unknown').replace('', 'Unknown')
    complaints['urgency'] = complaints['urgency'].fillna('Medium').replace('', 'Medium')

    if complaints.empty:
        print("⚠ No complaints to analyze.")
        return pd.DataFrame()

    # Scoring
    complaints['urgency_score'] = complaints['urgency'].map({'High': 3, 'Medium': 2, 'Low': 1}).fillna(0)
    complaints['frequency_score'] = complaints.groupby(['state', 'aspect'])['aspect'].transform('count')

    total_complaints = len(complaints)
    complaints['frequency_score'] = complaints['frequency_score'] / total_complaints if total_complaints > 0 else 0
    complaints['rating_impact_score'] = (5 - complaints['Rating']) / 4

    complaints['priority_score'] = (
        0.5 * complaints['urgency_score'] +
        0.3 * complaints['frequency_score'] +
        0.2 * complaints['rating_impact_score']
    ).round(2)

    # KPI and impact mappings
    kpi_mapping = {
        'Service': {'target': 'CSAT ≥ 90%', 'action': 'Barista training'},
        'Product': {'target': 'Score ≥ 4.2', 'action': 'Weekly QC'},
        'Ambiance': {'target': 'Rating ≥ 4.5', 'action': 'Clean audits'},
        'Wait Time': {'target': 'Avg < 3.5min', 'action': 'Optimize schedule'},
        'Price': {'target': 'Score ≥ 4.0', 'action': 'Loyalty value packs'},
        'Cleanliness': {'target': '≥ 4.8', 'action': 'Enhanced cleaning'},
        'Availability': {'target': '≥ 98%', 'action': 'Stock coordination'},
        'Unknown': {'target': 'Reduce Unknown 50%', 'action': 'LLM Deep Dive'}
    }

    impact_map = {
        'Service': 'High', 'Product': 'High', 'Wait Time': 'High',
        'Price': 'Medium', 'Ambiance': 'Medium', 'Cleanliness': 'High',
        'Availability': 'Medium', 'Unknown': 'Low'
    }

    # Group by state and aspect to get priority
    priority_issues = (
        complaints.groupby(['state', 'aspect'])
        .agg(priority_score=('priority_score', 'mean'), count=('aspect', 'size'))
        .reset_index()
        .sort_values(['state', 'priority_score'], ascending=[True, False])
    )

    priority_issues['kpi_target'] = priority_issues['aspect'].map(lambda x: kpi_mapping.get(x, {}).get('target', 'N/A'))
    priority_issues['kpi_action'] = priority_issues['aspect'].map(lambda x: kpi_mapping.get(x, {}).get('action', 'Analyze further'))
    priority_issues['expected_impact'] = priority_issues['aspect'].map(lambda x: impact_map.get(x, 'Medium'))

    # Extract keywords
    keyword_summary = (
        complaints.groupby(['state', 'aspect'])['clean_review']
        .apply(lambda x: extract_top_keywords(x.tolist(), top_n=5))
        .reset_index(name='common_keywords')
    )

    sentiment_counts = (
        df_combined
        .groupby(['state', 'aspect', 'sentiment'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
        .rename(columns={
            'Positive': 'positive_count',
            'Negative': 'negative_count',
            'Mixed': 'mixed_count'
        })
    )

    # Merge all together
    final_report = (
        priority_issues
        .merge(keyword_summary, on=['state', 'aspect'], how='left')
        .merge(sentiment_counts, on=['state', 'aspect'], how='left')
    )

    print("\nTop Priority Issues per State:")
    print(final_report.head(10))

    return final_report


In [71]:
priority_issues = generate_insights(df_final)
priority_issues.to_csv("priority_issues_summary.csv", index=False, encoding="utf-8-sig")


--- Generating Insights and Visualizations ---

Top Priority Issues per State:
  state        aspect  priority_score  count          kpi_target  \
0    CA       Service        1.360833     13          CSAT ≥ 90%   
1    CA       Unknown        1.306087     28  Reduce Unknown 50%   
2    CA      Ambiance        1.200000      1        Rating ≥ 4.5   
3    CA       Product        1.160000      3         Score ≥ 4.2   
4    CA         Price        1.123750      9         Score ≥ 4.0   
5    CA  Availability        0.710000      2               ≥ 98%   
6    FL       Product        1.710000      2         Score ≥ 4.2   
7    FL     Wait Time        1.650000      1        Avg < 3.5min   
8    FL       Service        1.360000      7          CSAT ≥ 90%   
9    FL       Unknown        1.289091     12  Reduce Unknown 50%   

            kpi_action expected_impact  \
0     Barista training            High   
1        LLM Deep Dive             Low   
2         Clean audits          Medium   
3  